In [1]:
import numpy as np
import pandas as pd

class display(object):
    """Display HTML representation of multiple objects"""
    template = """<div style="float: left; padding: 10px;">
    <p style='font-family:"Courier New", Courier, monospace'>{0}</p>{1}
    </div>"""
    def __init__(self, *args):
        self.args = args
        
    def _repr_html_(self):
        return '\n'.join(self.template.format(a, eval(a)._repr_html_())
                         for a in self.args)
    
    def __repr__(self):
        return '\n\n'.join(a + '\n' + repr(eval(a))
                           for a in self.args)

## Planets Data

In [2]:
import seaborn as sns
# import data
planets = sns.load_dataset('planets')
planets.shape

(1035, 6)

In [7]:
planets.head()

,method,number,orbital_period,mass,distance,year
0,Radial Velocity,1,269.300,7.10,77.40,2006
1,Radial Velocity,1,874.774,2.21,56.95,2008
2,Radial Velocity,1,763.000,2.60,19.84,2011
3,Radial Velocity,1,326.030,19.40,110.62,2007
4,Radial Velocity,1,516.220,10.50,119.47,2009


## Simple Aggregation in Pandas

In [9]:
rng = np.random.RandomState(42)
ser = pd.Series(rng.rand(5))
ser

0    0.374540
1    0.950714
2    0.731994
3    0.598658
4    0.156019
dtype: float64

In [10]:
ser.sum()

2.811925491708157

In [11]:
ser.mean()

0.5623850983416314

For a `DataFrame`, by default the aggregations return results within each column

In [12]:
df = pd.DataFrame({'A':rng.rand(5),
                   'B':rng.rand(5)})
df

,A,B
0,0.155995,0.020584
1,0.058084,0.969910
2,0.866176,0.832443
3,0.601115,0.212339
4,0.708073,0.181825


In [14]:
df.mean()

A    0.477888
B    0.443420
dtype: float64

In [16]:
np.mean(df)

A    0.477888
B    0.443420
dtype: float64

By specifying the `axis` argument, we can instead aggregate within each row

In [17]:
df.mean(axis=1)

0    0.088290
1    0.513997
2    0.849309
3    0.406727
4    0.444949
dtype: float64

Pandas `Series` and `DataFrame` include all of the common aggregates of Numpy's aggregations.  
In addition, there is a convenience method `describe()` that computes several common aggregates for each column.

In [28]:
planets.dropna().describe()

,number,orbital_period,mass,distance,year
count,498.00000,498.000000,498.000000,498.000000,498.000000
mean,1.73494,835.778671,2.509320,52.068213,2007.377510
std,1.17572,1469.128259,3.636274,46.596041,4.167284
min,1.00000,1.328300,0.003600,1.350000,1989.000000
25%,1.00000,38.272250,0.212500,24.497500,2005.000000
50%,1.00000,357.000000,1.245000,39.940000,2009.000000
75%,2.00000,999.600000,2.867500,59.332500,2011.000000
max,6.00000,17337.500000,25.000000,354.000000,2014.000000


The following table summarizes some other built-in Pandas aggregations:

| Aggregation              | Description                     |
|--------------------------|---------------------------------|
| ``count()``              | Total number of items           |
| ``first()``, ``last()``  | First and last item             |
| ``mean()``, ``median()`` | Mean and median                 |
| ``min()``, ``max()``     | Minimum and maximum             |
| ``std()``, ``var()``     | Standard deviation and variance |
| ``mad()``                | Mean absolute deviation         |
| ``prod()``               | Product of all items     
|
| ``sum()``                | Sum of all items                |

These are all methods of ``DataFrame`` and ``Series`` objects.

## GroupBy: Split, Apply, Combine

### Split, apply, combine

![](../PythonDataScienceHandbook/notebooks/figures/03.08-split-apply-combine.png)

This makes clear what the `groupby` accomplishes: 
+ The **split** step involves breaking up and grouping a `DataFrame` depending on the value of the specified key.
+ The **apply** step involves computing some function, usually an aggregate, transformation, or filltering, within the individual groups
+ The **conbine** step merges the results of these operations into an output array.

In [30]:
df = pd.DataFrame({'key': ['A', 'B', 'C', 'A', 'B', 'C'],
                   'data': range(6)}, columns=['key', 'data'])
df

,key,data
0,A,0
1,B,1
2,C,2
3,A,3
4,B,4
5,C,5


The most basic split-apply-combine operation can be computed with the `groupby()` method of `DataFrame`, passing the name of the desired key column

In [61]:
df.groupby('key')

The result is not a set of `DataFrame`s, but a `DataFrameGroupBy` object.  
We can think of it as a special view of the `DataFrame`, which is poised to dig into the groups but does not actual computation **until the aggregation is applied**  
To produce a result, we can apply an aggregate to it, which will perform the appropriate apply/combine steps to produce the desired result:

In [34]:
df.groupby('key').sum()

,data
key,
A,3
B,5
C,7


### The GroupBy object
Actually, we can treat the `GroupBy` object as it's a collection of `DataFrame`. and does not care how it done under the hood(it's difficult to express)

### Column indexing

The `GroupBy` object supports column indexing in the same way as the `DataFrame`, and return s a modified `GroupBy` object

In [40]:
planets.groupby('method')

In [44]:
planets.groupby('method')['orbital_period']

In [50]:
planets.groupby('method')['orbital_period'].median()

method
Astrometry                         631.180000
Eclipse Timing Variations         4343.500000
Imaging                          27500.000000
Microlensing                      3300.000000
Orbital Brightness Modulation        0.342887
Pulsar Timing                       66.541900
Pulsation Timing Variations       1170.000000
Radial Velocity                    360.200000
Transit                              5.714932
Transit Timing Variations           57.011000
Name: orbital_period, dtype: float64

#### Iteration over groups
The `GroupBy` object supports direct iteration over the groups, returning each group as a `Series` or `DataFrame`

In [64]:
for method, group in planets.groupby('method'):
    print("{:30s} shape={}".format(method, group.shape))

Astrometry                     shape=(2, 6)
Eclipse Timing Variations      shape=(9, 6)
Imaging                        shape=(38, 6)
Microlensing                   shape=(23, 6)
Orbital Brightness Modulation  shape=(3, 6)
Pulsar Timing                  shape=(5, 6)
Pulsation Timing Variations    shape=(1, 6)
Radial Velocity                shape=(553, 6)
Transit                        shape=(397, 6)
Transit Timing Variations      shape=(4, 6)


#### Dispatch methods

In [72]:
planets.groupby('method')['year'].describe()

,count,mean,std,min,25%,50%,75%,max
method,,,,,,,,
Astrometry,2.0,2011.500000,2.121320,2010.0,2010.75,2011.5,2012.25,2013.0
Eclipse Timing Variations,9.0,2010.000000,1.414214,2008.0,2009.00,2010.0,2011.00,2012.0
Imaging,38.0,2009.131579,2.781901,2004.0,2008.00,2009.0,2011.00,2013.0
Microlensing,23.0,2009.782609,2.859697,2004.0,2008.00,2010.0,2012.00,2013.0
Orbital Brightness Modulation,3.0,2011.666667,1.154701,2011.0,2011.00,2011.0,2012.00,2013.0
Pulsar Timing,5.0,1998.400000,8.384510,1992.0,1992.00,1994.0,2003.00,2011.0
Pulsation Timing Variations,1.0,2007.000000,NaN,2007.0,2007.00,2007.0,2007.00,2007.0
Radial Velocity,553.0,2007.518987,4.249052,1989.0,2005.00,2009.0,2011.00,2014.0
Transit,397.0,2011.236776,2.077867,2002.0,2010.00,2012.0,2013.00,2014.0


### Aggregate, filter, transform, apply

In [74]:
rng = np.random.RandomState(0)
df = pd.DataFrame({'key': ['A', 'B', 'C', 'A', 'B', 'C'],
                   'data1': range(6),
                   'data2': rng.randint(0, 10, 6)},
                   columns = ['key', 'data1', 'data2'])
df

,key,data1,data2
0,A,0,5
1,B,1,0
2,C,2,3
3,A,3,3
4,B,4,7
5,C,5,9


### Aggregation

In [80]:
df.groupby('key').aggregate(['min', np.std, 'max'])

data1              data2              
      min      std max   min       std max
key                                       
A       0  2.12132   3     3  1.414214   5
B       1  2.12132   4     0  4.949747   7
C       2  2.12132   5     3  4.242641   9

Another pattern is to pass a dictionary mapping column names to operations

In [85]:
df.groupby('key').aggregate({"data1":max,
                             "data2":[min, np.std]})

data1 data2          
      max   min       std
key                      
A       3     3  1.414214
B       4     0  4.949747
C       5     3  4.242641

#### Filltering
the filter function should return a Boolean value specifying whether the group passes the filtering

In [88]:
def filter_func(x):
    return x['data2'].std() > 4

display('df', "df.groupby('key').std()", "df.groupby('key').filter(filter_func)")

df
  key  data1  data2
0   A      0      5
1   B      1      0
2   C      2      3
3   A      3      3
4   B      4      7
5   C      5      9

df.groupby('key').std()
       data1     data2
key                   
A    2.12132  1.414214
B    2.12132  4.949747
C    2.12132  4.242641

df.groupby('key').filter(filter_func)
  key  data1  data2
1   B      1      0
2   C      2      3
4   B      4      7
5   C      5      9

#### Transformation

Transformation can return some transformed version of the full data to recombine. For such a transformation, the output is the same shape as the input

In [90]:
df.groupby('key').transform(lambda x: x - x.mean())

,data1,data2
0,-1.5,1.0
1,-1.5,-3.5
2,-1.5,-3.0
3,1.5,-1.0
4,1.5,3.5
5,1.5,3.0


In [95]:
def printer(x):
    return x+1
df.groupby('key').transform(printer)

,data1,data2
0,1,6
1,2,1
2,3,4
3,4,4
4,5,8
5,6,10


#### Apply() method

The `apply()` method lets you apply an arbitrary function to the group results. The function should take a DataFrame, and return either a Pandas object (e.g., DataFrame, Series) or a scalar; the combine operation will be tailored to the type of output returned.

In [105]:
def norm_by_data2(x):
    # x is a DataFrame of group values
#     x['data1'] /= x['data2'].sum()
    x += x.mean()
    return x
display('df', "df.groupby('key').apply(norm_by_data2)")

df
  key  data1  data2
0   A      0      5
1   B      1      0
2   C      2      3
3   A      3      3
4   B      4      7
5   C      5      9

df.groupby('key').apply(norm_by_data2)
   data1  data2
0    1.5    9.0
1    3.5    3.5
2    5.5    9.0
3    4.5    7.0
4    6.5   10.5
5    8.5   15.0

### Specifying the split key

#### A list, array, series, or index providing the groups key

In [107]:
# the key can be any series or list 
# with a length matching that of the DataFrame
L = [0, 1, 0, 1, 2, 0]
display('df', 'df.groupby(L).sum()')

df
  key  data1  data2
0   A      0      5
1   B      1      0
2   C      2      3
3   A      3      3
4   B      4      7
5   C      5      9

df.groupby(L).sum()
   data1  data2
0      7     17
1      4      3
2      4      7

In [115]:
l = np.random.randint(2,4, 6)
# print(l)
l = ['a','b','a','c','a','a']
display('df', 'df.groupby(l).sum()')

df
  key  data1  data2
0   A      0      5
1   B      1      0
2   C      2      3
3   A      3      3
4   B      4      7
5   C      5      9

df.groupby(l).sum()
   data1  data2
a     11     24
b      1      0
c      3      3

In [117]:
display('df', 'df.groupby(df["key"]).sum()')

df
  key  data1  data2
0   A      0      5
1   B      1      0
2   C      2      3
3   A      3      3
4   B      4      7
5   C      5      9

df.groupby(df["key"]).sum()
     data1  data2
key              
A        3      8
B        5      7
C        7     12

#### A dictionary or series mapping index to group

Another method is to provide a dictionary that maps index values to the group keys

In [126]:
df2 = df.set_index('key')
mapping = {'A':'vowel', 'B':'consonant', 'C':'consonant'}
df2.groupby(mapping).sum()

,data1,data2
consonant,12,19
vowel,3,8


#### Any python function

Similar to mapping, we can pass any Python function that will input the index value and output the group

In [127]:
display('df2', 'df2.groupby(str.lower).mean()')

,data1,data2
key,,
A,0,5
B,1,0
C,2,3
A,3,3
B,4,7
C,5,9
,data1,data2
a,1.5,4.0
b,2.5,3.5


#### A list of valid keys
Any of the preceding key choice can be combined to group on a multi-index

In [131]:
df2.groupby([str.lower, mapping]).mean()

,,data1,data2
a,vowel,1.5,4.0
b,consonant,2.5,3.5
c,consonant,3.5,6.0


### Grouping example

In [149]:
decade = 10 * (planets['year'] // 10)
decade = decade.astype(str) + 's'
decade.name = 'decade'
planets.groupby(['method', decade])['number'].sum().unstack().fillna(0)

decade,1980s,1990s,2000s,2010s
method,,,,
Astrometry,0.0,0.0,0.0,2.0
Eclipse Timing Variations,0.0,0.0,5.0,10.0
Imaging,0.0,0.0,29.0,21.0
Microlensing,0.0,0.0,12.0,15.0
Orbital Brightness Modulation,0.0,0.0,0.0,5.0
Pulsar Timing,0.0,9.0,1.0,1.0
Pulsation Timing Variations,0.0,0.0,1.0,0.0
Radial Velocity,1.0,52.0,475.0,424.0
Transit,0.0,0.0,64.0,712.0
